In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StringType, StructType, StructField
from pyspark.conf import SparkConf
conf = (
SparkConf()
    .setAppName("MY_kafka") # replace with your desired name
    .set("spark.jars.packages", \
         "org.apache.commons:commons-pool2:2.11.1"\
         ",org.apache.kafka:kafka-clients:3.4.0"\
         ",org.apache.spark:spark-protobuf_2.12:3.4.1"\
         ",org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1"\
         ",org.apache.spark:spark-token-provider-kafka-0-10_2.12:3.4.1"\
        )
    .setMaster("local")
    # .setMaster("spark://34.125.136.103:30077")
    # .set("spark.driver.host", "34.125.75.149") # 원격 실행 시 exited with code 1 exitStatus 1(무한재시작) 현상 방지용
    # .set("spark.driver.bindAddress", "0.0.0.0") # 원격 실행 시 exited with code 1 exitStatus 1(무한재시작) 현상 방지용
)

In [2]:
# Spark 세션 초기화
spark = SparkSession.builder \
    .config(conf=conf)\
    .getOrCreate()
properties = spark.sparkContext.getConf().getAll()
print(properties)

:: loading settings :: url = jar:file:/root/miniconda3/envs/spark341/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.commons#commons-pool2 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
org.apache.spark#spark-protobuf_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-token-provider-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-54432560-3158-4c3b-a3be-ffcff8fd8d81;1.0
	confs: [default]
	found org.apache.commons#commons-pool2;2.11.1 in central
	found org.apache.kafka#kafka-clients;3.4.0 in central
	found com.github.luben#zstd-jni;1.5.2-1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.36 in central
	found org.apache.spark#spark-protobuf_2.12;3.4.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.spark#spark-sql-kafka-0-10_2

[('spark.master', 'local'), ('spark.app.name', 'MY_kafka'), ('spark.app.submitTime', '1714673341386'), ('spark.app.initial.file.urls', 'file:///root/.ivy2/jars/org.apache.hadoop_hadoop-client-api-3.3.4.jar,file:///root/.ivy2/jars/org.spark-project.spark_unused-1.0.0.jar,file:///root/.ivy2/jars/com.google.code.findbugs_jsr305-3.0.0.jar,file:///root/.ivy2/jars/org.apache.hadoop_hadoop-client-runtime-3.3.4.jar,file:///root/.ivy2/jars/com.github.luben_zstd-jni-1.5.2-1.jar,file:///root/.ivy2/jars/org.xerial.snappy_snappy-java-1.1.10.1.jar,file:///root/.ivy2/jars/commons-logging_commons-logging-1.1.3.jar,file:///root/.ivy2/jars/org.apache.commons_commons-pool2-2.11.1.jar,file:///root/.ivy2/jars/org.lz4_lz4-java-1.8.0.jar,file:///root/.ivy2/jars/org.apache.spark_spark-token-provider-kafka-0-10_2.12-3.4.1.jar,file:///root/.ivy2/jars/org.apache.spark_spark-sql-kafka-0-10_2.12-3.4.1.jar,file:///root/.ivy2/jars/org.apache.spark_spark-protobuf_2.12-3.4.1.jar,file:///root/.ivy2/jars/org.apache.kafk

In [3]:

# Kafka 서버 정보
# host명으로 접근하기 위해 kube coredns에 hosts 등록해줘야 함
# kafka_bootstrap_servers = 'peter-kafka01.foo.bar:9092,peter-kafka02.foo.bar:9092,peter-kafka03.foo.bar:9092'
kafka_bootstrap_servers = 'pkc-lq8v7.eu-central-1.aws.confluent.cloud:9092'
topic = 'kafka-test'

# Kafka 스트림 읽기 설정
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
    .option("subscribe", topic) \
    .option("startingOffsets", "earliest") \
    .option("maxOffsetsPerTrigger", 100) \
    .load()

# # 메시지의 value를 JSON으로 파싱
# schema = StructType([
#     StructField("ident", StringType(), True),
#     StructField("type", StringType(), True),
#     StructField("name", StringType(), True),
#     StructField("elevation_ft", StringType(), True),
#     StructField("continent", StringType(), True),
#     StructField("iso_country", StringType(), True),
#     StructField("iso_region", StringType(), True),
#     StructField("municipality", StringType(), True),
#     StructField("gps_code", StringType(), True),
#     StructField("iata_code", StringType(), True),
#     StructField("local_code", StringType(), True),
#     StructField("coordinates", StringType(), True),
# ])

# json_df = df.select(from_json(col("value").cast("string"), schema).alias("data")).select("data.*")

In [ ]:
# 스트림 출력 설정
query = df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination()

24/05/03 03:09:10 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-6cc352c1-f1b2-4f5b-b3db-b4ce5caf07e6. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/05/03 03:09:10 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/05/03 03:09:11 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


In [ ]:
spark.stop()